In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
#plt.style.use('seaborn') #ggplot

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [ ]:
training = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/train.csv', na_values='?')
training


In [ ]:
training.isna().sum()

In [ ]:
training.head()

In [ ]:
training.info()

In [ ]:
training.describe()

In [ ]:
training.corr()

In [ ]:
plt.figure(figsize=(20,15))
sns.heatmap(training.corr(), annot=True)

In [ ]:
feature_corr = training.corr().unstack().sort_values()
feature_corr

In [ ]:
#Show Highly Correlated Features > 0.7
print(feature_corr[(abs(feature_corr)>0.7) & (abs(feature_corr)<1)].drop_duplicates())

In [ ]:
training = training.drop(columns=['id'])

In [ ]:
num_feature = training.columns[training.dtypes!='object']

def my_plot(feature):
    plt.hist(training[feature])
    plt.title(feature)
    plt.show()
        
for i in num_feature:
    my_plot(i)

In [ ]:
training.skew()

In [ ]:
training2 = training.copy()
features_to_transform = training2.loc[:,['cont0','cont4','cont5','cont6','cont7','cont8']]

In [ ]:
def features_log(feature):
    training2[feature] = np.log(training2[feature]+1)
        
for i in features_to_transform.columns:
    features_log(i)

In [ ]:
num_feature2 = training2.columns[training2.dtypes!='object']

def my_plot2(feature):
    plt.hist(training2[feature])
    plt.title(feature)
    plt.show()
        
for i in num_feature2:
    my_plot2(i)

In [ ]:
training2.skew()

In [ ]:
training_data_clear = pd.get_dummies(training2, drop_first=True)
training_data_clear.head()

In [ ]:
training_data_clear.info()

In [ ]:
x = training_data_clear.drop('target', axis=1)
y = pd.DataFrame(training_data_clear['target'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)

In [ ]:
print(x_train.shape)
x_train.head()

In [ ]:
print(y_train.shape)
y_train.head()

In [ ]:
scaler_x = MinMaxScaler().fit(x_train)
scaler_y = MinMaxScaler().fit(y_train)
x_train_sc = scaler_x.transform(x_train)
x_test_sc = scaler_x.transform(x_test)
y_train_sc = scaler_y.transform(y_train)
y_test_sc = scaler_y.transform(y_test)

In [ ]:
lr_model = LinearRegression()
lr_model.fit(x_train_sc, y_train_sc)
y_pred_sc = lr_model.predict(x_test_sc)

In [ ]:
mae = mean_absolute_error(y_test_sc, y_pred_sc)
rmse = np.sqrt(mean_squared_error(y_test_sc, y_pred_sc))

print('MAE = ', mae.round(4))
print('RMSE = ', rmse.round(4))

In [ ]:
y_test_inv = scaler_y.inverse_transform(y_test_sc.reshape(-1,1))
y_pred_inv = scaler_y.inverse_transform(y_pred_sc.reshape(-1,1))

actual_mae = mean_absolute_error(y_test_inv, y_pred_inv)
actual_rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))

print('Actual MAE = ', int(actual_mae))
print('Actual RMSE = ', int(actual_rmse))

In [ ]:
testing_data = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/test.csv', na_values='?')
testing_data

In [ ]:
testing_data_clear = testing_data.drop(columns=['id'])
testing_data_clear

In [ ]:
testing_data_clear.skew()

In [ ]:
testing_features_to_transform = testing_data_clear.loc[:,['cont0','cont4','cont5','cont6','cont7','cont8']]


In [ ]:
def features_log(feature):
    testing_data_clear[feature] = np.log(testing_data_clear[feature]+1)
        
for i in testing_features_to_transform.columns:
    features_log(i)
    
testing_data.skew()

In [ ]:
testing_data_clear = pd.get_dummies(testing_data_clear, drop_first=True)
testing_data_clear.head()

In [ ]:
testing_data_clear.insert(loc=30,
          column='cat6_G',
          value=0)
testing_data_clear.columns

In [ ]:
testing_data_scaled =scaler_x.transform(testing_data_clear)
testing_data_pred_sc = lr_model.predict(testing_data_scaled)
testing_data_pred_inv = scaler_y.inverse_transform(testing_data_pred_sc.reshape(-1,1))

In [ ]:
output = pd.DataFrame()
output['id']=testing_data.id
output['target']=testing_data_pred_inv
output.to_csv('submission.csv', index=False)